In [45]:
!pip install beautifulsoup4

In [68]:
import requests

url = 'https://accounts.spotify.com/api/token'

client_id = '52a1ea54f82c4769b64ec830b9951b73'
client_secret = '3b36b19509854042b130059ee22718cd' 

payload = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.post(url, headers=headers, data=payload)

data = response.json()
access_token = data["access_token"]

In [69]:
import requests

url = 'https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl'

headers = {
    'Authorization': f'Bearer {access_token}',
}

response = requests.get(url, headers=headers)

data = response.json()
print(data)

{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6sFIWsNpZYqfjUpaCgueju'}, 'href': 'https://api.spotify.com/v1/artists/6sFIWsNpZYqfjUpaCgueju', 'id': '6sFIWsNpZYqfjUpaCgueju', 'name': 'Carly Rae Jepsen', 'type': 'artist', 'uri': 'spotify:artist:6sFIWsNpZYqfjUpaCgueju'}], 'available_markets': [], 'external_urls': {'spotify': 'https://open.spotify.com/album/0tGPJ0bkWOUmH7MEOR77qc'}, 'href': 'https://api.spotify.com/v1/albums/0tGPJ0bkWOUmH7MEOR77qc', 'id': '0tGPJ0bkWOUmH7MEOR77qc', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b2737359994525d219f64872d3b1', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e027359994525d219f64872d3b1', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d000048517359994525d219f64872d3b1', 'width': 64}], 'name': 'Cut To The Feeling', 'release_date': '2017-05-26', 'release_date_precision': 'day', 'total_tracks': 1, 'type': 'album'

In [48]:
import os
print(os.getcwd())


/home/fipulab/notebooks/lblaskovic/music_cnn_classification


In [70]:
import requests
import os
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup

def fetch_playlist(playlist_id, access_token):
    api_url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(api_url, headers=headers)
    return response.json()

def download_preview(preview_url, save_path):
    print(f"Downloading audio from {preview_url} to {save_path}")
    try:
        # Download the actual audio file directly from the preview URL
        audio_response = requests.get(preview_url, stream=True)
        if audio_response.status_code == 200:
            with open(save_path, 'wb') as file:
                for chunk in audio_response.iter_content(1024):
                    file.write(chunk)
            print(f"Downloaded preview to {save_path}")
        else:
            print(f"Failed to download audio: HTTP {audio_response.status_code}")
    except Exception as e:
        print(f"Error in downloading audio from {preview_url} to {save_path}: {e}")

def save_previews(playlist_id, playlist_name, genre, access_token):
    playlist_data = fetch_playlist(playlist_id, access_token)
    print("Fetched playlist data")

    base_path = os.path.join(os.getcwd(), f'spotify/{genre}')

    if not os.path.exists(base_path):
        os.makedirs(base_path)
    print(f"Saving previews to {base_path}")

    download_tasks = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        for item in playlist_data['tracks']['items']:
            track = item['track']
            preview_url = track['preview_url']
            if preview_url:
                file_name = f"{track['name']}.mp3"
                save_path = os.path.join(base_path, file_name)
                task = executor.submit(download_preview, preview_url, save_path)
                download_tasks.append(task)

    for task in download_tasks:
        task.result()  # Wait for all downloads to complete
save_previews('3xxBCfDZ0w4cgX3bVDc8vN', 'metal', "metal", access_token)


Fetched playlist data
Saving previews to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/World on Fire.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Blasphemy.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Halo.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Fear Of The Dark - 1998 Remastered Version.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Avalon.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Duality.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Psychosocial.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spo

Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/The Night.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Absolute Zero.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/The Dark.mp3
Error in downloading audio from https://p.scdn.co/mp3-preview/609d8fc916aa8c870d5dd026e081be57f708037d?cid=52a1ea54f82c4769b64ec830b9951b73 to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/30/30-150.mp3: [Errno 2] No such file or directory: '/home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/30/30-150.mp3'
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Numb.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify/metal/Do Me a Favor.mp3
Downloaded preview to /home/fipulab/notebooks/lblaskovic/music_cnn_classification/spotify

KeyboardInterrupt: 

## Problem - these clips are mp3 !
- got very bad accuracy on existing models

In [71]:
!pip install pydub

In [75]:
from pydub import AudioSegment
import os

genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
source_dir = 'spotify'
destination_dir = 'wav'

os.makedirs(destination_dir, exist_ok=True)

for genre in genres:
    genre_path = os.path.join(source_dir, genre)
    genre_destination_path = os.path.join(destination_dir, genre)

    os.makedirs(genre_destination_path, exist_ok=True)

    if os.path.exists(genre_path):
        for file in os.listdir(genre_path):
            if file.endswith('.mp3'):
                file_path = os.path.join(genre_path, file)
                wav_file_name = file.replace('.mp3', '.wav')
                wav_file_path = os.path.join(genre_destination_path, wav_file_name)

                if not os.path.exists(wav_file_path):
                    print(f"Converting {file_path} to {wav_file_path}...")
                    song = AudioSegment.from_mp3(file_path)
                    song.export(wav_file_path, format='wav')


Converting spotify/blues/Junker's Blues.mp3 to wav/blues/Junker's Blues.wav...
Converting spotify/blues/The Sky Is Crying.mp3 to wav/blues/The Sky Is Crying.wav...
Converting spotify/blues/Feels Like Rain (feat. Bonnie Raitt).mp3 to wav/blues/Feels Like Rain (feat. Bonnie Raitt).wav...
Converting spotify/blues/Mean Old World.mp3 to wav/blues/Mean Old World.wav...
Converting spotify/blues/Albert's Shuffle.mp3 to wav/blues/Albert's Shuffle.wav...
Converting spotify/blues/Stumble.mp3 to wav/blues/Stumble.wav...
Converting spotify/blues/Damn Right, I've Got the Blues.mp3 to wav/blues/Damn Right, I've Got the Blues.wav...
Converting spotify/blues/All Your Love.mp3 to wav/blues/All Your Love.wav...
Converting spotify/blues/Double Trouble.mp3 to wav/blues/Double Trouble.wav...
Converting spotify/blues/Love Me Like a Man - 2008 Remaster.mp3 to wav/blues/Love Me Like a Man - 2008 Remaster.wav...
Converting spotify/blues/Call It Stormy Monday.mp3 to wav/blues/Call It Stormy Monday.wav...
Convert

Converting spotify/classical/Clair De Lune.mp3 to wav/classical/Clair De Lune.wav...
Converting spotify/classical/Sonata in C Major, No. 3: IV. Allegro.mp3 to wav/classical/Sonata in C Major, No. 3: IV. Allegro.wav...
Converting spotify/classical/Mendelssohns Lieder, S. 547: Auf Flugeln des Gesanges (from Op. 34, No. 2).mp3 to wav/classical/Mendelssohns Lieder, S. 547: Auf Flugeln des Gesanges (from Op. 34, No. 2).wav...
Converting spotify/classical/Debussy: 2 Arabesques, CD 74, L. 66: No. 1, Andantino con moto.mp3 to wav/classical/Debussy: 2 Arabesques, CD 74, L. 66: No. 1, Andantino con moto.wav...
Converting spotify/classical/Sylvia: Act III: Divertissement: Variation - Valse.mp3 to wav/classical/Sylvia: Act III: Divertissement: Variation - Valse.wav...
Converting spotify/classical/Lyric Pieces, Book 1, Op. 12: Arietta.mp3 to wav/classical/Lyric Pieces, Book 1, Op. 12: Arietta.wav...
Converting spotify/classical/Idea 9.mp3 to wav/classical/Idea 9.wav...
Converting spotify/classical/

Converting spotify/disco/High Energy.mp3 to wav/disco/High Energy.wav...
Converting spotify/disco/Histoire d'1 soir (Bye bye les galères) [Version radio].mp3 to wav/disco/Histoire d'1 soir (Bye bye les galères) [Version radio].wav...
Converting spotify/disco/Gonna Get Over You.mp3 to wav/disco/Gonna Get Over You.wav...
Converting spotify/disco/Good Times - 7" Edit; 2018 Remaster.mp3 to wav/disco/Good Times - 7" Edit; 2018 Remaster.wav...
Converting spotify/disco/Yes Sir, I Can Boogie.mp3 to wav/disco/Yes Sir, I Can Boogie.wav...
Converting spotify/disco/Give Me the Night - Edit.mp3 to wav/disco/Give Me the Night - Edit.wav...
Converting spotify/disco/Boys - Summertime Love.mp3 to wav/disco/Boys - Summertime Love.wav...
Converting spotify/disco/Rivers of Babylon - Single Version.mp3 to wav/disco/Rivers of Babylon - Single Version.wav...
Converting spotify/disco/Can You Feel It - 7" Version.mp3 to wav/disco/Can You Feel It - 7" Version.wav...
Converting spotify/disco/Disco Inferno - Edit

Converting spotify/jazz/Body and Soul.mp3 to wav/jazz/Body and Soul.wav...
Converting spotify/jazz/Ruby My Dear.mp3 to wav/jazz/Ruby My Dear.wav...
Converting spotify/jazz/Freddie Freeloader (feat. John Coltrane, Cannonball Adderley, Wynton Kelly & Paul Chambers).mp3 to wav/jazz/Freddie Freeloader (feat. John Coltrane, Cannonball Adderley, Wynton Kelly & Paul Chambers).wav...
Converting spotify/jazz/My Baby Just Cares for Me (2013 - Remaster).mp3 to wav/jazz/My Baby Just Cares for Me (2013 - Remaster).wav...
Converting spotify/jazz/Goodbye Pork Pie Hat.mp3 to wav/jazz/Goodbye Pork Pie Hat.wav...
Converting spotify/jazz/Mack the Knife.mp3 to wav/jazz/Mack the Knife.wav...
Converting spotify/jazz/It Ain't Necessarily So.mp3 to wav/jazz/It Ain't Necessarily So.wav...
Converting spotify/jazz/Moanin'.mp3 to wav/jazz/Moanin'.wav...
Converting spotify/jazz/jazz is for ordinary people.mp3 to wav/jazz/jazz is for ordinary people.wav...
Converting spotify/metal/The Last Fight.mp3 to wav/metal/Th

Converting spotify/pop/Lose Control.mp3 to wav/pop/Lose Control.wav...
Converting spotify/pop/Baby Blue.mp3 to wav/pop/Baby Blue.wav...
Converting spotify/pop/Blessing.mp3 to wav/pop/Blessing.wav...
Converting spotify/pop/FLORIDA.mp3 to wav/pop/FLORIDA.wav...
Converting spotify/pop/BIG GIRL BANGER.mp3 to wav/pop/BIG GIRL BANGER.wav...
Converting spotify/pop/Dance The Night - From Barbie The Album.mp3 to wav/pop/Dance The Night - From Barbie The Album.wav...
Converting spotify/pop/My House.mp3 to wav/pop/My House.wav...
Converting spotify/pop/Sympathy.mp3 to wav/pop/Sympathy.wav...
Converting spotify/pop/All About That Bass.mp3 to wav/pop/All About That Bass.wav...
Converting spotify/pop/Candy Store.mp3 to wav/pop/Candy Store.wav...
Converting spotify/pop/Everybody Falls (5yr).mp3 to wav/pop/Everybody Falls (5yr).wav...
Converting spotify/pop/Selfish.mp3 to wav/pop/Selfish.wav...
Converting spotify/pop/PILLOWTALK.mp3 to wav/pop/PILLOWTALK.wav...
Converting spotify/pop/Love Song.mp3 to w

Converting spotify/rock/Back In Black.mp3 to wav/rock/Back In Black.wav...
Converting spotify/rock/Uprising.mp3 to wav/rock/Uprising.wav...
Converting spotify/rock/Mississippi Queen.mp3 to wav/rock/Mississippi Queen.wav...
Converting spotify/rock/Breaking the Law.mp3 to wav/rock/Breaking the Law.wav...
Converting spotify/rock/House Of The Rising Sun.mp3 to wav/rock/House Of The Rising Sun.wav...
Converting spotify/rock/Money.mp3 to wav/rock/Money.wav...
Converting spotify/rock/Don't Stop Believin' (2022 Remaster).mp3 to wav/rock/Don't Stop Believin' (2022 Remaster).wav...
Converting spotify/rock/Barracuda.mp3 to wav/rock/Barracuda.wav...
Converting spotify/rock/Another Brick in the Wall, Pt. 2.mp3 to wav/rock/Another Brick in the Wall, Pt. 2.wav...
Converting spotify/rock/Hey Joe.mp3 to wav/rock/Hey Joe.wav...
Converting spotify/rock/Californication.mp3 to wav/rock/Californication.wav...
Converting spotify/rock/Jump - 2015 Remaster.mp3 to wav/rock/Jump - 2015 Remaster.wav...
Converting

## wav to mp3

In [76]:
from pydub import AudioSegment
import os

genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
source_dir = 'dataset/GTZAN/genres_original'
destination_dir = 'dataset/GTZAN_mp3'

os.makedirs(destination_dir, exist_ok=True)

for genre in genres:
    genre_path = os.path.join(source_dir, genre)
    genre_destination_path = os.path.join(destination_dir, genre)

    os.makedirs(genre_destination_path, exist_ok=True)

    if os.path.exists(genre_path):
        for file in os.listdir(genre_path):
            if file.endswith('.wav'):
                file_path = os.path.join(genre_path, file)
                mp3_file_name = file.replace('.wav', '.mp3')
                mp3_file_path = os.path.join(genre_destination_path, mp3_file_name)

                if not os.path.exists(mp3_file_path):
                    print(f"Converting {file_path} to {mp3_file_path}...")
                    song = AudioSegment.from_wav(file_path)
                    song.export(mp3_file_path, format='mp3')

Converting dataset/GTZAN/genres_original/blues/blues.00060.wav to dataset/GTZAN_mp3/blues/blues.00060.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00048.wav to dataset/GTZAN_mp3/blues/blues.00048.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00043.wav to dataset/GTZAN_mp3/blues/blues.00043.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00071.wav to dataset/GTZAN_mp3/blues/blues.00071.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00064.wav to dataset/GTZAN_mp3/blues/blues.00064.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00039.wav to dataset/GTZAN_mp3/blues/blues.00039.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00038.wav to dataset/GTZAN_mp3/blues/blues.00038.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00034.wav to dataset/GTZAN_mp3/blues/blues.00034.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00029.wav to dataset/GTZAN_mp3/blues/blues.00029.mp3...
Converting dataset/

Converting dataset/GTZAN/genres_original/blues/blues.00052.wav to dataset/GTZAN_mp3/blues/blues.00052.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00063.wav to dataset/GTZAN_mp3/blues/blues.00063.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00041.wav to dataset/GTZAN_mp3/blues/blues.00041.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00020.wav to dataset/GTZAN_mp3/blues/blues.00020.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00035.wav to dataset/GTZAN_mp3/blues/blues.00035.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00061.wav to dataset/GTZAN_mp3/blues/blues.00061.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00025.wav to dataset/GTZAN_mp3/blues/blues.00025.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00009.wav to dataset/GTZAN_mp3/blues/blues.00009.mp3...
Converting dataset/GTZAN/genres_original/blues/blues.00080.wav to dataset/GTZAN_mp3/blues/blues.00080.mp3...
Converting dataset/

Converting dataset/GTZAN/genres_original/classical/classical.00082.wav to dataset/GTZAN_mp3/classical/classical.00082.mp3...
Converting dataset/GTZAN/genres_original/classical/classical.00038.wav to dataset/GTZAN_mp3/classical/classical.00038.mp3...
Converting dataset/GTZAN/genres_original/classical/classical.00041.wav to dataset/GTZAN_mp3/classical/classical.00041.mp3...
Converting dataset/GTZAN/genres_original/classical/classical.00051.wav to dataset/GTZAN_mp3/classical/classical.00051.mp3...
Converting dataset/GTZAN/genres_original/classical/classical.00052.wav to dataset/GTZAN_mp3/classical/classical.00052.mp3...
Converting dataset/GTZAN/genres_original/classical/classical.00006.wav to dataset/GTZAN_mp3/classical/classical.00006.mp3...
Converting dataset/GTZAN/genres_original/classical/classical.00092.wav to dataset/GTZAN_mp3/classical/classical.00092.mp3...
Converting dataset/GTZAN/genres_original/classical/classical.00001.wav to dataset/GTZAN_mp3/classical/classical.00001.mp3...


Converting dataset/GTZAN/genres_original/country/country.00076.wav to dataset/GTZAN_mp3/country/country.00076.mp3...
Converting dataset/GTZAN/genres_original/country/country.00072.wav to dataset/GTZAN_mp3/country/country.00072.mp3...
Converting dataset/GTZAN/genres_original/country/country.00036.wav to dataset/GTZAN_mp3/country/country.00036.mp3...
Converting dataset/GTZAN/genres_original/country/country.00026.wav to dataset/GTZAN_mp3/country/country.00026.mp3...
Converting dataset/GTZAN/genres_original/country/country.00063.wav to dataset/GTZAN_mp3/country/country.00063.mp3...
Converting dataset/GTZAN/genres_original/country/country.00065.wav to dataset/GTZAN_mp3/country/country.00065.mp3...
Converting dataset/GTZAN/genres_original/country/country.00015.wav to dataset/GTZAN_mp3/country/country.00015.mp3...
Converting dataset/GTZAN/genres_original/country/country.00066.wav to dataset/GTZAN_mp3/country/country.00066.mp3...
Converting dataset/GTZAN/genres_original/country/country.00082.w

Converting dataset/GTZAN/genres_original/country/country.00020.wav to dataset/GTZAN_mp3/country/country.00020.mp3...
Converting dataset/GTZAN/genres_original/country/country.00098.wav to dataset/GTZAN_mp3/country/country.00098.mp3...
Converting dataset/GTZAN/genres_original/country/country.00037.wav to dataset/GTZAN_mp3/country/country.00037.mp3...
Converting dataset/GTZAN/genres_original/country/country.00073.wav to dataset/GTZAN_mp3/country/country.00073.mp3...
Converting dataset/GTZAN/genres_original/country/country.00030.wav to dataset/GTZAN_mp3/country/country.00030.mp3...
Converting dataset/GTZAN/genres_original/country/country.00074.wav to dataset/GTZAN_mp3/country/country.00074.mp3...
Converting dataset/GTZAN/genres_original/country/country.00041.wav to dataset/GTZAN_mp3/country/country.00041.mp3...
Converting dataset/GTZAN/genres_original/country/country.00052.wav to dataset/GTZAN_mp3/country/country.00052.mp3...
Converting dataset/GTZAN/genres_original/country/country.00000.w

...

Converting dataset/GTZAN/genres_original/metal/metal.00003.wav to dataset/GTZAN_mp3/metal/metal.00003.mp3...
Converting dataset/GTZAN/genres_original/metal/metal.00030.wav to dataset/GTZAN_mp3/metal/metal.00030.mp3...
Converting dataset/GTZAN/genres_original/metal/metal.00034.wav to dataset/GTZAN_mp3/metal/metal.00034.mp3...
Converting dataset/GTZAN/genres_original/metal/metal.00010.wav to dataset/GTZAN_mp3/metal/metal.00010.mp3...
Converting dataset/GTZAN/genres_original/metal/metal.00008.wav to dataset/GTZAN_mp3/metal/metal.00008.mp3...
Converting dataset/GTZAN/genres_original/metal/metal.00070.wav to dataset/GTZAN_mp3/metal/metal.00070.mp3...
Converting dataset/GTZAN/genres_original/metal/metal.00063.wav to dataset/GTZAN_mp3/metal/metal.00063.mp3...
Converting dataset/GTZAN/genres_original/metal/metal.00050.wav to dataset/GTZAN_mp3/metal/metal.00050.mp3...
Converting dataset/GTZAN/genres_original/metal/metal.00079.wav to dataset/GTZAN_mp3/metal/metal.00079.mp3...
Converting dataset/

Converting dataset/GTZAN/genres_original/pop/pop.00055.wav to dataset/GTZAN_mp3/pop/pop.00055.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00054.wav to dataset/GTZAN_mp3/pop/pop.00054.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00020.wav to dataset/GTZAN_mp3/pop/pop.00020.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00040.wav to dataset/GTZAN_mp3/pop/pop.00040.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00088.wav to dataset/GTZAN_mp3/pop/pop.00088.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00098.wav to dataset/GTZAN_mp3/pop/pop.00098.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00059.wav to dataset/GTZAN_mp3/pop/pop.00059.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00014.wav to dataset/GTZAN_mp3/pop/pop.00014.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00002.wav to dataset/GTZAN_mp3/pop/pop.00002.mp3...
Converting dataset/GTZAN/genres_original/pop/pop.00086.wav to dataset/GTZAN_mp3/pop/pop.000

Converting dataset/GTZAN/genres_original/reggae/reggae.00055.wav to dataset/GTZAN_mp3/reggae/reggae.00055.mp3...
Converting dataset/GTZAN/genres_original/reggae/reggae.00058.wav to dataset/GTZAN_mp3/reggae/reggae.00058.mp3...
Converting dataset/GTZAN/genres_original/reggae/reggae.00076.wav to dataset/GTZAN_mp3/reggae/reggae.00076.mp3...
Converting dataset/GTZAN/genres_original/reggae/reggae.00034.wav to dataset/GTZAN_mp3/reggae/reggae.00034.mp3...
Converting dataset/GTZAN/genres_original/reggae/reggae.00090.wav to dataset/GTZAN_mp3/reggae/reggae.00090.mp3...
Converting dataset/GTZAN/genres_original/reggae/reggae.00092.wav to dataset/GTZAN_mp3/reggae/reggae.00092.mp3...
Converting dataset/GTZAN/genres_original/reggae/reggae.00033.wav to dataset/GTZAN_mp3/reggae/reggae.00033.mp3...
Converting dataset/GTZAN/genres_original/reggae/reggae.00032.wav to dataset/GTZAN_mp3/reggae/reggae.00032.mp3...
Converting dataset/GTZAN/genres_original/reggae/reggae.00010.wav to dataset/GTZAN_mp3/reggae/reg

Converting dataset/GTZAN/genres_original/rock/rock.00049.wav to dataset/GTZAN_mp3/rock/rock.00049.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00037.wav to dataset/GTZAN_mp3/rock/rock.00037.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00051.wav to dataset/GTZAN_mp3/rock/rock.00051.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00008.wav to dataset/GTZAN_mp3/rock/rock.00008.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00078.wav to dataset/GTZAN_mp3/rock/rock.00078.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00036.wav to dataset/GTZAN_mp3/rock/rock.00036.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00024.wav to dataset/GTZAN_mp3/rock/rock.00024.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00043.wav to dataset/GTZAN_mp3/rock/rock.00043.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00097.wav to dataset/GTZAN_mp3/rock/rock.00097.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.0001

Converting dataset/GTZAN/genres_original/rock/rock.00062.wav to dataset/GTZAN_mp3/rock/rock.00062.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00075.wav to dataset/GTZAN_mp3/rock/rock.00075.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00077.wav to dataset/GTZAN_mp3/rock/rock.00077.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00064.wav to dataset/GTZAN_mp3/rock/rock.00064.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00094.wav to dataset/GTZAN_mp3/rock/rock.00094.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00005.wav to dataset/GTZAN_mp3/rock/rock.00005.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00031.wav to dataset/GTZAN_mp3/rock/rock.00031.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00081.wav to dataset/GTZAN_mp3/rock/rock.00081.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.00040.wav to dataset/GTZAN_mp3/rock/rock.00040.mp3...
Converting dataset/GTZAN/genres_original/rock/rock.0007

# Resample function

In [ ]:
import librosa

def change_sample_rate(audio_data, original_rate, target_rate):

    # Resample the audio data using librosa
    resampled_data = librosa.resample(audio_data, orig_sr=original_rate, target_sr=target_rate)
    return resampled_data

In [78]:
import os
import librosa
import soundfile as sf
from concurrent.futures import ThreadPoolExecutor

def resample_audio_file(file_path, genre_source_path, genre_destination_path, target_rate):
    if os.path.isfile(file_path):
        audio_data, original_rate = librosa.load(file_path, sr=None)

        if original_rate != target_rate:
            audio_data = librosa.resample(audio_data, orig_sr=original_rate, target_sr=target_rate)

        destination_file_path = os.path.join(genre_destination_path, os.path.basename(file_path))
        sf.write(destination_file_path, audio_data, target_rate)

def resample_audio(source_dir, destination_dir, genres, target_rate):

    with ThreadPoolExecutor() as executor:
        for genre in genres:
            genre_source_path = os.path.join(source_dir, genre)
            genre_destination_path = os.path.join(destination_dir, genre)
            os.makedirs(genre_destination_path, exist_ok=True)

            futures = []
            for filename in os.listdir(genre_source_path):
                file_path = os.path.join(genre_source_path, filename)
                futures.append(executor.submit(resample_audio_file, file_path, genre_source_path, genre_destination_path, target_rate))

            for future in futures:
                future.result()

genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
# Define source and destination directories
source_dir = 'spotify'
destination_dir = 'spotify_resampled'

target_sample_rate = 22050

resample_audio(source_dir, destination_dir, genres, target_sample_rate)
